### Doing
 - [ ] Add CMS_IDs to Outbreaks and Facilities

### To Do
 - [ ] Group Facilities by CMS_Provider_Num WHERE Available
 
### DONE
 - [x] Separate funtions into separate Python files


In [1]:
import pandas as pd
import urllib3 as urllib
import urllib.request as urllib2
import json
import glob
import IPython.display
import altair as alt

from Modules.IL_Functions import *

pd.options.display.max_columns = None

## 1 - Get "Raw Data" from website and store in file

In [2]:
[reporting_date, ltc_data] = pull_IL_json_from_web()

MaxRetryError: HTTPSConnectionPool(host='idph.illinois.gov', port=443): Max retries exceeded with url: /DPHPublicInformation/api/covid/getltcdata (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x0000020AB1748608>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))

In [ ]:
# print('Source File: ' + str(json_file))
[reporting_date, summary, outbreak_df, df_facilities, df_county] = process_IL_dict(ltc_data, ltc_name2cms_id, display_dfs=False)

In [ ]:
# outbreak_df.groupby(['County', 'FacilityName']).sum()

In [ ]:
df_facilities.head(5)

In [ ]:
df_facilities.sort_values(by='Open_Outbreaks', ascending=False).head(20)

In [ ]:
outbreak_df.head(5)

In [ ]:
df_facilities.head(10)

In [ ]:
# # display(df_facilities.reset_index().head(5))
# df_facilities.head(5)

In [ ]:
df_facilities.reset_index().groupby(['CMS_ProvNum']).count().sort_values(by='County', ascending=False)

In [ ]:
df_facilities = df_facilities.reset_index()
s = df_facilities.CMS_ProvNum.value_counts()
dupNamesList = list(s.where((s > 1) & (s < 100)).dropna().keys())

In [ ]:
s = df_facilities.CMS_ProvNum.value_counts()
dupNamesList = list(s.where((s > 1) & (s < 100)).dropna().keys())

In [ ]:
dupFacNames_df = df_facilities[df_facilities['CMS_ProvNum'].isin(dupNamesList)].sort_values(by='CMS_ProvNum')
dupFacNames_df.to_csv('IL_dup_fac_names.csv')
dupFacNames_df

# Convert Facility Sheet to CTP format

In [ ]:
CTP_cols = ['ReportingDate', 'State', 'County', 
            'City',
            'FacilityName', 'TYPE',
            'CTP Facility Categorization',
            'State/Fed Regulated',
            'State Facility ID',
            'CMS Facility ID',
            'Date outbreak opened', 'Date outreak closed', 'Outbreak Status','Resident Census',
            
'TOTAL RESIDENT CASES', # 'Resident Positives ',
          'TOTAL Resident Probable Positives',
'TOTAL RESIDENT DEATHS', # 'Resident Deaths',
         'TOTAL Resident Probable Deaths',
'TOTAL STAFF CASES', #'Staff Positive',
         'TOTAL Staff Probable Positives',
'TOTAL STAFF DEATHS', #'TOTAL Staff Deaths',
         'TOTAL Staff Probable Deaths',
 'confirmed_cases',
 'TOTAL Resident/Staff Probable Positives',
 'deaths',
 'TOTAL Resident/Staff Probable Deaths',
 
'NEW RESIDENT CASES', #            'Resident Positives',
     'NEW Resident Probable Positives',
 'NEW RESIDENT DEATHS',
     'NEW Resident Probable Deaths',
 'NEW STAFF CASES',
     'NEW Staff Probable Positives',
 'NEW STAFF DEATHS',
     'NEW Staff Probable Deaths',
 'NEW Resident/Staff Positives',
 'NEW Resident/Staff Probable Positives',
 'NEW Resident/Staff Deaths',
 'NEW Resident/Staff Probable Deaths',
 'Residents Tested',
 'Staff Tested',
 'Personal Protective Equipment',
           'COMPLIANCE']

CTP_Blank_Cols = ['City',
             'TYPE',
            'CTP Facility Categorization',
            'State/Fed Regulated',
            'State Facility ID',
            'CMS Facility ID',
            'Date outbreak opened', 'Date outreak closed', 'Resident Census',
            
'TOTAL RESIDENT CASES', # 'Resident Positives ',
          'TOTAL Resident Probable Positives',
'TOTAL RESIDENT DEATHS', # 'Resident Deaths',
         'TOTAL Resident Probable Deaths',
'TOTAL STAFF CASES', #'Staff Positive',
         'TOTAL Staff Probable Positives',
'TOTAL STAFF DEATHS', #'TOTAL Staff Deaths',
         'TOTAL Staff Probable Deaths',

 'TOTAL Resident/Staff Probable Positives',

 'TOTAL Resident/Staff Probable Deaths',
 
'NEW RESIDENT CASES', #            'Resident Positives',
     'NEW Resident Probable Positives',
 'NEW RESIDENT DEATHS',
     'NEW Resident Probable Deaths',
 'NEW STAFF CASES',
     'NEW Staff Probable Positives',
 'NEW STAFF DEATHS',
     'NEW Staff Probable Deaths',
 'NEW Resident/Staff Positives',
 'NEW Resident/Staff Probable Positives',
 'NEW Resident/Staff Deaths',
 'NEW Resident/Staff Probable Deaths',
 'Residents Tested',
 'Staff Tested',
 'Personal Protective Equipment',
           'COMPLIANCE']



In [ ]:
def open_closed(outbreaks):
    if outbreaks == 0:
        return 'Closed'
    else:
        return 'Open'
        
for col in CTP_Blank_Cols:
    df_facilities[col] = ''
df_facilities['Outbreak Status'] = df_facilities['Open_Outbreaks'].apply(open_closed) #TO FINISH
df_facilities['State'] = 'IL'
df_facilities['ReportingDate'] = reporting_date.replace('-', '')

In [ ]:
df_facilities.rename(columns={'FacilityName2': 'FacilityName'}, inplace=True)
df_facilities[CTP_cols].to_csv('Reporting_data/IL_' + reporting_date + '_Facilities_LTC_data_vCTP.csv', index=False)

In [ ]:
df_facilities.head()

# Play Area

In [ ]:
df = pd.DataFrame(ltc_data['LTC_Historical_Reported_Cases'])
df.sort_values(by=['reported_date'], inplace=True)
df['new_deaths']= df.deaths.diff()
df['new_confirmed_cases'] = df.confirmed_cases.diff()

In [ ]:
df.head(5)

In [ ]:
chart_title = 'IL LTC Cases by Week'
chart1 = alt.Chart(df).mark_bar().encode(
                     x=alt.X('yearmonthdate(reported_date):O', title='date', axis=alt.Axis(labelSeparation=10)),
                     y='new_confirmed_cases',
                     ).properties( width=400, height=400, title=chart_title)
chart_title = 'IL LTC Deaths by Week'
chart2 = alt.Chart(df).mark_bar().encode(
                     x=alt.X('yearmonthdate(reported_date):O', title='date', axis=alt.Axis(labelSeparation=10)),
                     y='new_deaths',
                     ).properties( width=400, height=400, title=chart_title)
chart1 | chart2

In [ ]:
il_data_df = pd.read_json('https://api.covidtracking.com/v1/states/il/daily.json')
il_data_df['date']= pd.to_datetime(il_data_df['date'].astype(str), format='%Y%m%d')

In [ ]:
chart_title = 'IL Cases by Week'
il_data_df['legend'] = 'state'
il_data_df['positiveIncrease_7day'] = il_data_df['positiveIncrease'].rolling(7).mean().round(1)
chart3 = alt.Chart(il_data_df).mark_line(color="#FFAA00").encode(
                     x=alt.X('yearmonthdate(date):O', title='date', axis=alt.Axis(labelSeparation=10)),
                     y='positiveIncrease_7day',
                     color=alt.Color('legend', legend=alt.Legend(title="Cases"))
                     ).properties( width=400, height=400, title=chart_title)

chart_title = 'IL Cases by Week'
df['legend'] = 'LTC'

chart1 = alt.Chart(df).mark_line().encode(
                     x=alt.X('yearmonthdate(reported_date):O', title='date', axis=alt.Axis(labelSeparation=10)),
                     y='new_confirmed_cases',
                     color=alt.Color('legend', legend=alt.Legend(title="Cases"))
                     ).properties( width=400, height=400, title=chart_title)
chart1 + chart3